In [9]:
%matplotlib
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from HikeClass import Hike
from KinematicClass import Kinematic
from tqdm import tqdm

from video.proccess import image_proccess, grouping, roi_center, metric_reference

Using matplotlib backend: TkAgg


In [2]:
# path1 = '/home/mariano/Descargas/VID_20170720_132629833.mp4'  # Belen
# path2 = '/home/mariano/Descargas/VID_20170720_133000367.mp4'  # Belen corriendo
# path3 = '/home/mariano/Descargas/VID_20170728_112226772.mp4'  # Maxi Hissa

# path1 = '/home/mariano/Descargas/VID_20170831_093451442.mp4'  # Ferrando v
# path2 = '/home/mariano/Descargas/VID_20170831_095319235.mp4'  # Ferrando o
path3 = '/home/mariano/Descargas/VID_20170831_100017607.mp4'  # Ferrando l

# path1 = '/home/mariano/Escritorio/marcha/Carla Minxo/VID_20170906_133539527.mp4'
# path2 = '/home/mariano/Escritorio/marcha/Carla Minxo/VID_20170906_134024147.mp4'
# path3 = '/home/mariano/Escritorio/marcha/Carla Minxo/VID_20170906_134919420.mp4'

kinematics = []
for path in (path3, ):
    vid = cv2.VideoCapture(path)
    fps = vid.get(cv2.CAP_PROP_FPS)
    frames_num = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    expected_markers = (2, 2, 3)
    M = sum(expected_markers)
    safe_None = 1.5 * fps
    last_M = 0
    active = False
    count_non_data = 0
    trayectories = []
    n_metrics = 5
    metrics = np.zeros((1, 1))

    t = Hike()
    for i in tqdm(xrange(frames_num)):
        __, frame = vid.read()
        m, mark = image_proccess(frame)
        if mark.any():
            if m == M:
                active = True
                if not t.start_videoframe_position:
                    t.start_videoframe_position = i
                last_M = i
                count_non_data = 0
    ############################################################################################################
    # Se adiciona código para obtener metricas.
    # Esta parte del codigo es peligrosa.. debe hacerce notar la importancia de inciar con metricas cada video.
            if (not metrics.any() and not active) and i < 50:  # DEBUG HARDCORE
                if m == n_metrics:
                    metrics = mark
    ############################################################################################################
            if active:
                if m != M:
                    to_interp, groups = grouping(mark, expected_markers, kr)
                    t.add_markers_from_videoframe(i, groups)
                    t.add_index_to_interpolate(i, to_interp)
                else:
                    __, groups = grouping(mark, expected_markers)
                    kr = roi_center(groups[1])
                    t.add_markers_from_videoframe(i, groups)
        else:
            count_non_data += 1
            if active and count_non_data < safe_None:
                G0 = np.random.random(size=(2, 2))
                G1 = np.random.random(size=(2, 2))
                G2 = np.random.random(size=(3, 2))
                t.add_markers_from_videoframe(i, (G0, G1, G0))
                t.add_index_to_interpolate(i, (True, True, True))
            else:
                if active:
                    backwards = (i - 1) - last_M
                    t.rm_last_data_added(backwards)
                    t.end_videoframe_position = i - backwards - 1
                    trayectories.append(t)
                    t = Hike()
                    active = False

    ref = metric_reference(metrics)
#     kinematics.append(Kinematic(path, fps, ref, trayectories))

100%|██████████| 3190/3190 [00:57<00:00, 55.63it/s]


In [7]:
H, K = trayectories[:2]
H.joints_definition()
K.joints_definition()
hf = H.joints_as_dataframe()
kf = H.joints_as_dataframe('B')

In [37]:
kf = kf.reorder_levels(['joint', 'cycle'])

In [4]:
# color = {None:'', 1:'b', -1:'g'}
# for H in trayectories:
#     H.get_foot_direction()
#     H.fix_groups()
#     H.joints_definition()
#     plt.plot(H.joints[0]['ankle'], color=color[H.direction])

In [5]:
# from time import sleep

# H = trayectories[3]
# X, Y = H.markers_as_dataframe('fixed').loc['pa'].values
# M = H.start_videoframe_position
# m = H.end_videoframe_position

# indexes = H.cycles
# vid = cv2.VideoCapture(path)
# n = 0
# i = 0
# while True:
#     slp = .001
#     is_frame, frame = vid.read()
#     if is_frame is False:
#         break
#     if n >= M and n < m:
#         cv2.circle(frame, (X[i], Y[i]), 7, (0, 110, 225), -1)
#         if i in indexes:
#             slp = 1
#         i += 1
#     n += 1
#     cv2.imshow(path, cv2.resize(frame, None, fx=.4, fy=.4))
#     k = cv2.waitKey(30) & 0xff
#     if k == 27:
#         break
#     sleep(slp)
# vid.release()